In [27]:
import pandas as pd
from datetime import datetime, date, timedelta
from ulticlipper.user_defned_classes.game import Game
from ulticlipper.user_defned_classes.event import Event, EventType, LineType
from ulticlipper.user_defned_classes.player import Player
import numpy as np
import string


file = "./ulticlipper/test_data/RaleighFlyers2019-stats.csv"
game_date = datetime(2019,4,5)
tournament = "AUDL"
opponent = "Dallas Roughnecks"
video_offset_s = 6

In [28]:
def get_team_data(file:str) -> tuple[pd.DataFrame, list[list[str,str,str]]]:
    csv = pd.read_csv(file)
    csv.rename(columns={"Tournamemnt":"Tournament"}, inplace=True)
    unique = csv.groupby(['Date/Time','Tournament', 'Opponent'])['Date/Time','Tournament', 'Opponent'].apply(lambda x: list(np.unique(x)))
    csv['Date/Time']= pd.to_datetime(csv['Date/Time'])

    return csv, unique.values.tolist()

csv, unique_games = get_team_data(file)

print(unique_games)
csv['Elapsed Time (secs)']

[['2019-04-05 19:24', 'AUDL', 'Dallas Roughnecks'], ['2019-04-06 18:06', 'AUDL', 'Austin Sol'], ['2019-04-13 19:00', 'AUDL', 'Atlanta Hustle'], ['2019-04-27 19:11', 'AUDL', 'Atlanta Hustle'], ['2019-05-05 13:12', 'AUDL', 'Austin Sol'], ['2019-05-11 19:05', 'AUDL', 'Tampa Bay Cannons'], ['2019-05-18 19:10', 'AUDL', 'Atlanta Hustle'], ['2019-06-15 19:05', 'AUDL', 'Dallas Roughnecks'], ['2019-06-22 19:03', 'AUDL', 'New York Empire'], ['2019-06-29 19:20', 'AUDL', 'Tampa Bay Cannons'], ['2019-07-05 19:47', 'AUDL', 'Dallas Roughnecks'], ['2019-07-13 19:01', 'AUDL', 'Tampa Bay Cannons'], ['2019-07-27 19:03', 'AUDL', 'Dallas Roughnecks']]


C:\Users\jcarr\AppData\Local\Temp\ipykernel_5468\3104703083.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  unique = csv.groupby(['Date/Time','Tournament', 'Opponent'])['Date/Time','Tournament', 'Opponent'].apply(lambda x: list(np.unique(x)))


0          0
1         26
2         40
3         45
4         47
        ... 
4306    6528
4307    6534
4308    6537
4309    6594
4310    6632
Name: Elapsed Time (secs), Length: 4311, dtype: int64

In [29]:
# filtering for one game
def get_game_data(csv, tournament: str, opponent: str, game_date: datetime, video_offset_s:int) -> pd.DataFrame:
    game_data = csv[
        (csv["Tournament"] == tournament)
        & (csv["Opponent"] == opponent)
        & (
            (csv["Date/Time"] >= game_date)
            & (csv["Date/Time"] <= (game_date + timedelta(hours = 24)))
        )
    ].copy()
    game_data["Elapsed Time (secs)"] = game_data["Elapsed Time (secs)"] + video_offset_s
    return game_data

game_data = get_game_data(csv, tournament, opponent, game_date, video_offset_s)

game_data

,Date/Time,Tournament,Opponent,Point Elapsed Seconds,Line,Our Score - End of Point,Their Score - End of Point,Event Type,Action,Passer,...,Begin Area,Begin X,Begin Y,End Area,End X,End Y,Distance Unit of Measure,Absolute Distance,Lateral Distance,Toward Our Goal Distance
1797,2019-04-05 19:24:00,AUDL,Dallas Roughnecks,156,O,0,1,Offense,Catch,LavioleA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1798,2019-04-05 19:24:00,AUDL,Dallas Roughnecks,156,O,0,1,Offense,Catch,TaylorEr,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1799,2019-04-05 19:24:00,AUDL,Dallas Roughnecks,156,O,0,1,Offense,Catch,MunizCha,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1800,2019-04-05 19:24:00,AUDL,Dallas Roughnecks,156,O,0,1,Offense,Catch,FisherHe,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1801,2019-04-05 19:24:00,AUDL,Dallas Roughnecks,156,O,0,1,Offense,Drop,MitchelT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2078,2019-04-05 19:24:00,AUDL,Dallas Roughnecks,143,D,17,19,Defense,Goal,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2079,2019-04-05 19:24:00,AUDL,Dallas Roughnecks,121,O,17,19,Offense,Catch,TaylorEr,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2080,2019-04-05 19:24:00,AUDL,Dallas Roughnecks,121,O,17,19,Offense,Catch,AllenKir,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2081,2019-04-05 19:24:00,AUDL,Dallas Roughnecks,121,O,17,19,Offense,Throwaway,MunizCha,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
# get data objects from data
def get_game_objects(game_data: pd.DataFrame) -> Game:

    # getting data to use for creating game object
    first_row = game_data.iloc[0]
    last_row = game_data.iloc[-1]

    opponent = first_row["Opponent"]
    tournament = first_row["Tournament"]
    datetime_of_game = first_row["Date/Time"]
    starting_line_type = LineType[first_row["Line"]]
    their_final_score = last_row["Their Score - End of Point"]
    our_final_score = last_row["Our Score - End of Point"]
    elapsed_time_of_first_pull = int(first_row["Elapsed Time (secs)"])
    game_duration_s = int(last_row["Elapsed Time (secs)"]) - elapsed_time_of_first_pull

    active_players = set()
    events = []

    previous_event_start = int(first_row["Elapsed Time (secs)"])

    # looping through each event
    for index, row in game_data.iterrows():
        event_type = EventType[
            row["Action"]
            .translate(str.maketrans("", "", string.punctuation))
            .upper()
        ]
        line_type = LineType[row["Line"]]
        players_on = []
        for x in range(28):
            if pd.notnull(row[f"Player {x}"]):
                players_on.append(Player(row[f"Player {x}"]))
        active_players.update(players_on)
        event_start_elapsed = int(row["Elapsed Time (secs)"])
        event_start_datetime = datetime_of_game + timedelta(seconds=event_start_elapsed)
        passer = row["Passer"] if pd.notnull(row["Passer"]) else None
        reciever = row["Receiver"] if pd.notnull(row["Receiver"]) else None
        defender = row["Defender"] if pd.notnull(row["Defender"]) else None
        hang_time = float(row["Hang Time (secs)"]) if pd.notnull(row["Hang Time (secs)"]) else None

        events.append(
            Event(
                event_type=event_type,
                opponent=opponent,
                tournament=tournament,
                datetime_game=datetime_of_game,
                event_start_datetime=event_start_datetime,
                event_start_elapsed=event_start_elapsed,
                players_on=players_on,
                line_type=line_type,
                our_score=row["Our Score - End of Point"],
                their_score=row["Their Score - End of Point"],
                passer=passer,
                reciever=reciever,
                defender=defender,
                hang_time=hang_time,
            )
        )

    return Game(
        opponent=opponent, 
        tournament = tournament,
        active_players = list(active_players),
        datetime_of_first_pull = datetime_of_game,
        elapsed_time_of_first_pull = elapsed_time_of_first_pull, 
        starting_line_type = starting_line_type,
        our_final_score = our_final_score, 
        their_final_score = their_final_score, 
        duration_s = game_duration_s,
        events = events
    )

game_object = get_game_objects(game_data)

In [36]:
import time
previous_start = game_object.events[0].event_start_elapsed
time.sleep(previous_start)
for event in game_object.events[0:10]:
    print(event)
    time.sleep(event.event_start_elapsed-previous_start)
    previous_start = event.event_start_elapsed

EventType.CATCH: LavioleA TaylorEr None
EventType.CATCH: TaylorEr MunizCha None
EventType.CATCH: MunizCha FisherHe None
EventType.CATCH: FisherHe MitchelT None
EventType.DROP: MitchelT AllenKir None
EventType.THROWAWAY: None None Anonymous
EventType.CATCH: TaylorEr MunizCha None
EventType.CATCH: MunizCha AllenKir None
EventType.CATCH: AllenKir FairfaxJ None
EventType.CATCH: FairfaxJ TaylorEr None
